## search api

In [102]:
import urllib2
import urllib
import requests
from datetime import timedelta, datetime
from bs4 import BeautifulSoup
import cPickle as pickle
import re
import numpy as np

http://api.nytimes.com/svc/search/v2/articlesearch.json?q=books+review&fq=The+New+York+Times&begin_date=20150301&end_date=20150330&sort=newest&fl=web_url%2Cheadline%2Ckeywords&hl=true&api-key=38477d93e711afb3260ca02fcd5b1cdb%3A12%3A71777620

In [11]:
r = requests.get('http://api.nytimes.com/svc/search/v2/articlesearch.json?q=books+review&fq=The+New+York+Times&begin_date=20150301&end_date=20150330&sort=newest&fl=web_url%2Cheadline%2Ckeywords&hl=true&api-key=38477d93e711afb3260ca02fcd5b1cdb%3A12%3A71777620')
rjson = r.json()

In [14]:
type(rjson)

dict

## best sellers api

In [7]:
l = requests.get('http://api.nytimes.com/svc/books/v3/lists/names.json?api-key=ed91070ee5df688636a455cb3ad9cb72%3A2%3A71777620')

In [9]:
blist = l.json()
blist.keys()

[u'status', u'num_results', u'results', u'copyright']

In [10]:
rlist = blist['results']

In [11]:
def extractdict(l, k):
    return [x[k].encode('utf-8') for x in l]

In [239]:
bestlist = extractdict(rlist, 'display_name')
pickle.dump(bestlist, open('listname.p', 'wb'))
#bestlist = picke.load( open('listname.p', 'wb'))


Get female and male first name lists

In [16]:
import pandas as pd
fem = pd.read_table('http://deron.meranda.us/data/census-dist-female-first.txt', sep='[ ]+',  header=None)
ma = pd.read_table('http://deron.meranda.us/data/census-dist-male-first.txt', sep='[ ]+',  header=None) 

In [29]:
perfem = fem[1].tolist()
permale = ma[1].tolist()
female = fem[0].str.lower().tolist()
male = ma[0].str.lower().tolist()
female2=[]
inmale = []
for f in range(len(female)):
    if female[f] in male:
        inmale.append(female[f])
        topop = male.index(female[f])
        if perfem[f] > permale[topop]:
            male.pop(topop)
            permale.pop(topop)
            female2.append(female[f])
    else:
        female2.append(female[f])
female = female2
len(female) 

4014

Get literature genre

In [264]:
g = requests.get('http://en.wikipedia.org/wiki/List_of_literary_genres')
wg = g.text
gidx = wg.find('id="Genre')
aw = wg[gidx:wg.find('<h2>', gidx)]
l = re.findall('<li>(.+)</li>', aw)
#re.findall('title="(.+)"', aw)
genre = []
for i in l:
    if i[0] == '<':
        genre.append(re.findall('<a .+>(.+)</a>', i)[0].lower())
    else:
        genre.append(re.findall('(.+)&', i)[0].lower())
genre

[u'classic',
 u'graphic novel',
 u'detective',
 u'fable',
 u'fairy tale',
 u'fanfiction',
 u'fantasy',
 u'fiction narrative',
 u'fiction in verse',
 u'folklore',
 u'historical fiction',
 u'horror',
 u'humor',
 u'legend',
 u'magical realism',
 u'metafiction',
 u'mystery',
 u'mythology',
 u'mythopoeia',
 u'poetry',
 u'realistic fiction',
 u'science fiction',
 u'short story',
 u'thriller',
 u'tall tale',
 u'western',
 u'autobiography',
 u'essay',
 u'narrative nonfiction',
 u'speech',
 u'textbook',
 u'reference book']

http://api.nytimes.com/svc/books/v3/lists/2014-01-15/Hardcover%20Fiction.json?api-key=ed91070ee5df688636a455cb3ad9cb72%3A2%3A71777620

In [268]:
bestlist = pickle.load( open('listname.p', 'rb'))
hf = urllib.quote(bestlist[2])
hf

'Hardcover%20Fiction'

In [289]:
startDate = "01/01/09"
date = datetime.strptime(startDate, "%m/%d/%y")

In [275]:
def get_list(d, tl):
    baseurl = 'http://api.nytimes.com/svc/books/v3/lists/'
    endurl = '.json?api-key=ed91070ee5df688636a455cb3ad9cb72%3A2%3A71777620'     
    url = baseurl + str(date.year) + '-' + date.strftime('%m') + '-' + date.strftime('%d') + '/' + tl + endurl
    l = requests.get(url)
    return l.json()

In [287]:
#get_list(date, hf)

In [290]:
ybest = {}
for i in range(52*14):
    print date
    l= get_list(date, hf)
    if len(l) > 0:
        ybest[date.strftime('%y-%m-%d')] = l['results']['books']
    date = date + timedelta(days=7)

2009-01-01 00:00:00
2009-01-08 00:00:00
2009-01-15 00:00:00
2009-01-22 00:00:00
2009-01-29 00:00:00
2009-02-05 00:00:00
2009-02-12 00:00:00
2009-02-19 00:00:00
2009-02-26 00:00:00
2009-03-05 00:00:00
2009-03-12 00:00:00
2009-03-19 00:00:00
2009-03-26 00:00:00
2009-04-02 00:00:00
2009-04-09 00:00:00
2009-04-16 00:00:00
2009-04-23 00:00:00
2009-04-30 00:00:00
2009-05-07 00:00:00
2009-05-14 00:00:00
2009-05-21 00:00:00
2009-05-28 00:00:00
2009-06-04 00:00:00
2009-06-11 00:00:00
2009-06-18 00:00:00
2009-06-25 00:00:00
2009-07-02 00:00:00
2009-07-09 00:00:00
2009-07-16 00:00:00
2009-07-23 00:00:00
2009-07-30 00:00:00
2009-08-06 00:00:00
2009-08-13 00:00:00
2009-08-20 00:00:00
2009-08-27 00:00:00
2009-09-03 00:00:00
2009-09-10 00:00:00
2009-09-17 00:00:00
2009-09-24 00:00:00
2009-10-01 00:00:00
2009-10-08 00:00:00
2009-10-15 00:00:00
2009-10-22 00:00:00
2009-10-29 00:00:00
2009-11-05 00:00:00
2009-11-12 00:00:00
2009-11-19 00:00:00
2009-11-26 00:00:00
2009-12-03 00:00:00
2009-12-10 00:00:00


TypeError: list indices must be integers, not str

In [292]:
pickle.dump(ybest, open('ybest.p', 'wb'))
#ybest= pickle.load( open('ybest.p', 'rb'))


In [293]:
def get_wikilist(wiki, nm, wiurl):
    ''' this function takes t, he text of wikipedia and return a list composed 
    of the date of the awards or bibliography '''
    
    aidx = wiki.find('id="' + nm)
    if aidx == -1 and nm == 'Bibliography':
        match = re.search('id=".*[Ww]orks', wiki)
        widx = 0
        if match != None:
            widx = match.start()
        pidx = wiki.find('id="Publish')
        bidx = wiki.find('id="Books')
        nidx = wiki.find('id="Novels')
        aidx = widx + pidx + bidx + nidx
    if aidx == -1 and nm == 'Awards':
        match = re.search('[Pp]rize', wiki)
        
    aw = wiki[aidx:wiki.find('<h2>', aidx)]
    award = re.findall('<td.*>(.+)</td>', aw) + re.findall('<td>(.+)</td>', aw) + re.findall('<li>(.+)</li>', aw) + \
    re.findall('<dt>(.+)</dt>', aw) 
    if len(award) == 0 and nm == 'Bibliography':
        biurl = wiurl + '_bibliography'
        b = requests.get(biurl)
        if b.status_code == 404:
            return -1
        else:
            bibli = b.text
            aidx = bibli.find('id="Bibliography')
            aw = bibli[aidx:bibli.find('<h2>', aidx)]
            award = re.findall('<td>(.+)</td>', aw) + re.findall('<td.*>(.+)</td>', aw)+ \
            re.findall('<li>(.+)</li>', aw) + re.findall('<dt>(.+)</dt>', aw) 
            re.findall('<dt>(.+)</dt>', aw)
    elif len(award) == 0:
        return -1
    date = []
    for i in award:
        d = re.findall('[0-9]{4}', i)    
        if len(d) > 1:
            d = [d[0]]
        if len(d) > 0:        
            if d[0][0:2] != '19' and d[0][0:2] != '20':
                continue
            date.append(d[0])
    if len(date) == 0:
        return len(award)
    return len(date)

Find some info about the author and the number of published book and awards on wikipedia

In [294]:
def get_genre(wiki, genre):
    out = []
    for i in genre:
        if i in wiki:
            out.append(i)
    return out


In [298]:
basewi = 'http://en.wikipedia.org/wiki/'
def wikiscrape(basewi, onebook, female, male, genre):
    author = onebook['author']
    author = author.encode('utf-8')
    firstname = re.findall('\w+',author)[0]
    if firstname.lower() in female:
        g = 'F'
    elif firstname.lower() in male:
        g = 'M'
    else:
        g = '?'
    onebook['gender'] = g
    if len(re.findall('\w+',author)) > 2:
        author = re.findall('\w+',author)[0] + '_' + re.findall('\w+',author)[1]      
    author = author.replace(' ', '_')
    wiurl  = basewi + author
    w      = requests.get(wiurl)
    wiki   = w.text
    if wiki.find('Disambiguation icon') != -1:
        wiurl  = basewi + author + '_author'
        w      = requests.get(wiurl)
        wiki   = w.text
    if wiki.find('Wikipedia does not have an article with this exact name') != -1:
        onebook['genre'] = ''
        onebook['biblio'] = ''
        onebook['awards'] = ''
        onebook['age'] = ''
        onebook['country'] = ''
        return onebook
    blurb = re.findall('<p>(.+)</p>', wiki)[0]
    onebook['genre'] = get_genre(blurb, genre)
    onebook['biblio'] = get_wikilist(wiki, 'Bibliography', wiurl)
    onebook['awards'] = get_wikilist(wiki, 'Awards', wiurl)
    a = re.findall('class="bday"(.+)-', wiki)
    if len(a) > 0:
        a = re.findall('[0-9]{4}', a[0])[0]
        onebook['age'] = str(datetime.now().year - int(a.decode('utf8')))
    else:
        soup = BeautifulSoup(wiki)
        t = soup.find('table')
        if t != None:
            a = re.findall('[0-9]{4}', t.text)
            if len(a) > 0:
                a = a[0]
                if len(a) > 0:
                    onebook['age'] = str(datetime.now().year - int(a.decode('utf8')))
            else:
                onebook['age'] =''
        else:
            onebook['age'] =''
    c = re.findall('<td class="category".*>(.+)</td>', wiki)
    if len(c) > 0:
        country = c[0].encode('utf8').lower()
        usa = 'united states, america, american, us, u.s., usa, u.s.a'
        if usa.find(country) != -1:
            country = 'USA'
        onebook['country'] = country
    else:
        onebook['country'] = ''
    return onebook

In [124]:
m = ybest['14-01-01']
#best = [wikiscrape(basewi, x, female, male) for x in m]
#wikiscrape(basewi, m[10], female, male, genre)


In [299]:
best = {}
for k in ybest.keys():
    print k
    if len(ybest[k]) > 0:
        best[k] = [wikiscrape(basewi, x, female, male, genre) for x in ybest[k]]

09-04-09
14-08-28
10-08-26
11-08-11
13-02-14
09-04-02
11-07-14
09-09-10
09-09-17
09-06-25
12-03-22
10-07-01
09-01-01
13-01-03
09-03-19
15-01-01
09-01-08
10-11-04
15-01-08
12-07-26
09-03-12
10-07-22
11-09-29
14-09-25
11-03-24
09-02-05
13-04-25
13-03-07
14-03-20
10-07-29
15-03-26
11-01-06
15-02-05
13-07-18
09-08-06
09-07-23
13-05-30
13-10-24
10-10-07
13-07-11
14-12-04
10-12-16
11-02-24
10-09-23
15-03-05
15-04-16
10-08-12
10-01-21
11-07-07
11-09-08
09-09-24
09-05-07
10-01-28
09-04-30
10-08-19
12-02-23
12-04-12
14-05-01
11-02-03
09-01-15
15-03-19
12-04-19
11-03-10
11-08-18
15-03-12
11-03-17
11-05-12
10-05-06
10-09-09
09-10-08
15-01-15
14-02-20
09-10-01
14-12-11
09-07-30
13-05-23
11-06-16
14-10-02
10-10-14
11-06-02
11-11-24
13-11-28
11-08-25
14-05-08
10-09-30
12-03-15
09-04-23
12-03-01
14-10-30
12-11-22
14-06-12
12-06-28
11-02-10
12-03-08
09-05-14
14-06-19
11-02-17
15-02-12
10-05-20
09-01-22
12-04-05
10-05-27
11-05-05
12-07-05
11-03-03
09-01-29
11-01-20
13-04-04
14-07-10
09-10-15
11-09-22
1

In [300]:
pickle.dump(best, open('best.p', 'wb'))
#ybest= pickle.load( open('ybest.p', 'rb'))

Get the data in a dataframe

In [377]:
key = best[best.keys()[1]][0].keys()
nbook = len(best[best.keys()[1]])
lendf = len(best) * nbook
col[:] = []
col = key
col.insert(0,'dates')
df = pd.DataFrame(index=range(lendf), columns=col)
df.shape
b = best[best.keys()[1]]
v = b[1].values()
v.insert(0,i)

#df.loc[0] = v
#[x.encode('utf8') if type(x) == 'str' for x in v ]

[type(x) for x in v]
v[17] = str(v[17])
[type(x) for x in v]
v2 = ['' if type(x) == list else x for x in v ]
#..v3 = [x.encode('ascii') if type(x) == unicode else x for x in v2]
df.loc[0] = v
v3
df[0:1]
v3


UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 49: ordinal not in range(128)

In [316]:
k = 0
for i in best.keys():
    b = best[i]
    print i
    for j in range(nbook):
        print j
        v = b[j].values()
        v.insert(0,i)
        print len(v)
        df.loc[k] = v
        #print df
        k += 1

09-04-09
0
29


ValueError: setting an array element with a sequence

In [228]:
pickle.dump(df, open('df.p', 'wb'))
#ybest= pickle.load( open('ybest.p', 'rb'))


get all the reviews of the year

In [255]:
r = requests.get('http://api.nytimes.com/svc/search/v2/articlesearch.json?q=books+review&begin_date=20140101&end_date=20141231&api-key=38477d93e711afb3260ca02fcd5b1cdb%3A12%3A71777620')
rjson = r.json()

In [258]:
len(rjson)
rjson

{u'copyright': u'Copyright (c) 2013 The New York Times Company.  All Rights Reserved.',
 u'response': {u'docs': [{u'_id': u'53fbb76479881038210a164a',
    u'abstract': u'Martin Filler acknowledged a significant error in a New York Review of Books article he wrote that was harshly critical of Ms. Hadid.',
    u'blog': [],
    u'byline': {u'original': u'By ALLAN KOZINN',
     u'person': [{u'firstname': u'Allan',
       u'lastname': u'KOZINN',
       u'organization': u'',
       u'rank': 1,
       u'role': u'reported'}]},
    u'document_type': u'blogpost',
    u'headline': {u'kicker': u'ArtsBeat',
     u'main': u'Architecture Critic Acknowledges a Mistake in Critique of Zaha Hadid',
     u'print_headline': u'Architect&#8217;s Lawsuit Elicits Apology'},
    u'keywords': [{u'name': u'persons',
      u'rank': u'1',
      u'value': u'Filler, Martin'},
     {u'name': u'persons', u'rank': u'2', u'value': u'Hadid, Zaha'},
     {u'name': u'organizations',
      u'rank': u'1',
      u'value': u'Ne